<a href="https://colab.research.google.com/github/navidadkhah/Fine-Tuning-LLMs/blob/main/Dataset/Create_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing requirement libraries

In [2]:
!pip install mutmut
!pip install astor
!pip install libcst

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 kB 9.2 MB/s eta 0:00:00
  Created wheel for mutmut: filename=mutmut-2.5.1-py2.py3-none-any.whl size=31324 sha256=07bbc31b231cb87aa6fb80e7279cc1eeec60a436014d6ae918f40018f368cb21
  Stored in directory: /root/.cache/pip/wheels/38/a3/c7/572da9a8ddf88686624a4f70d8605b887086a98f8a02b1ebec
Successfully built mutmut
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import csv
import json
from collections.abc import Iterator
import ast    # To convert code to AST
import astor  # To convert AST back to code
import random  # To select mutations randomly
import libcst as cst
import libcst.matchers as m


# Reading Dataset

In [4]:
# Reading not completed dataset
path = '/content/drive/MyDrive/Bachelor\'s project/python_codes.csv'
df = pd.read_csv(path)
print(f"Number of Dataframe rows : {df.shape}")
df.head()

Number of Dataframe rows : (23187, 2)


,language,source
0,Python 3,"n, m = map(int, input().split())\na = []\nfor ..."
1,Python 3,"n, m = map(int, input().split())\na = []\nfor ..."
2,Python 3,#In the name of Allah\n\nfrom sys import stdin...
3,Python 3,"s = input()\nl, *v = (int(x) for x in input()...."
4,Python 3,"s = input()\nl, *v = (int(x) for x in input()...."


In [5]:
# Droping language column
df = df.drop('language', axis=1)
print(df.shape)
df.head()

(23187, 1)


,source
0,"n, m = map(int, input().split())\na = []\nfor ..."
1,"n, m = map(int, input().split())\na = []\nfor ..."
2,#In the name of Allah\n\nfrom sys import stdin...
3,"s = input()\nl, *v = (int(x) for x in input()...."
4,"s = input()\nl, *v = (int(x) for x in input()...."


In [6]:
# Removing codes cotain comments
def has_comments(code):
    return '#' in code

df = df[~df['source'].apply(has_comments)]
print(df.shape)
df = df.reset_index(drop=True)
df.head()

(17743, 1)


,source
0,"n, m = map(int, input().split())\na = []\nfor ..."
1,"n, m = map(int, input().split())\na = []\nfor ..."
2,"s = input()\nl, *v = (int(x) for x in input()...."
3,"s = input()\nl, *v = (int(x) for x in input()...."
4,"n = int(input())\na = list(map(int, input().sp..."


In [7]:
# Reducing the number of rows
desired_rows = 3000
selected_list = []
final_df = pd.DataFrame(columns=['source'])
while desired_rows != len(selected_list):
  rand_row = random.randrange(0, df.shape[0])
  if rand_row not in selected_list:
    selected_list.append(rand_row)
    final_df = pd.concat([final_df, pd.DataFrame(df['source'][rand_row:rand_row+1])])
df = final_df.reset_index(drop=True)
print(df.shape)
df.head()

(3000, 1)


,source
0,n=int(input())\ns=input()\nmax=0\nfor i in ran...
1,"import math\n\np, y = map(int, input().split()..."
2,from decimal import *\nimport math\ngetcontext...
3,"n = int(input())\nlove = list(map(int, input()..."
4,from fractions import gcd\nS = input()\nn = in...


In [8]:
df['source'][230].split("\n")

['you=1',
 'friend=10**6',
 "total=input('total:')",
 "number=input('each number:')",
 'index=list(map(int, number.strip().split()))',
 'A=friend-index[-1]',
 'B=index[-1]-1',
 'if A<B:',
 '    print(A)',
 'else:',
 '    print(B)']

# Mutanting code
In this section, we use mutation technique code using the CST library. Changing operations in the code and randomly replace with CST operations.
## Why we are using CST?
We are using CST instead of AST because we want to save the format of the structure too and CST does it for us.
<br>
In the output, we expect a mutant code and a line to tell us which line has changed.

In [282]:
import ast
import random

def mutate_code(source_code):
    tree = ast.parse(source_code)

    mutation_type = random.choice(["value", "decision", "statement"])


    print(mutation_type)
    if mutation_type == "value":
        # Value Mutation: Change integer constants
        for node in ast.walk(tree):
            if isinstance(node, ast.Constant) and isinstance(node.value, int):
                node.value = random.randint(1, 10000)
                break  # Apply mutation to only one line

    elif mutation_type == "decision":
        # Decision Mutation: Change comparison operators
        for node in ast.walk(tree):
            if isinstance(node, ast.Compare):
                node.ops[0] = random.choice([ast.Gt(), ast.Lt(), ast.Eq(), ast.GtE(), ast.LtE()])
                break  # Apply mutation to only one line

    elif mutation_type == "statement":
        # Statement Mutation: Change the assigned variable name
        for node in ast.walk(tree):
            if isinstance(node, ast.Assign) and isinstance(node.targets[0], ast.Name):
                node.targets[0].id = random.choice(['x', 'y', 'z'])
                break  # Apply mutation to only one line

    # Convert the AST back into source code
    mutated_code = astor.to_source(tree).strip()
    mutated_lines = mutated_code.split('\n')

    # Print the mutated code
    # print(mutated_code)

    return ast.unparse(tree)

# Example code to mutate
source_code = '''
from decimal import *
import math
getcontext().prec=50


a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))'''

# Mutate the code
mutated_code = mutate_code(source_code)
print(mutated_code)


TypeError: CodeMutator.leave_Assign() takes 2 positional arguments but 3 were given

In [268]:
# Change operation
import random
import libcst as cst

# Initialize the global variable
flag = False
op_list = []

class OperationChanger(cst.CSTTransformer):

    select_operations = [
        cst.Add, cst.Subtract, cst.Multiply, cst.Divide,
        cst.Modulo, cst.Power, cst.FloorDivide,
        cst.LeftShift, cst.RightShift,
        cst.BitOr, cst.BitXor, cst.BitAnd,
        cst.MatrixMultiply
    ]

    operations = [
        cst.Add(), cst.Subtract(), cst.Multiply(), cst.Divide(),
        cst.Modulo(), cst.Power(), cst.FloorDivide(),
        cst.LeftShift(), cst.RightShift(),
        cst.BitOr(), cst.BitXor(), cst.BitAnd(),
        cst.MatrixMultiply()
    ]

    def __init__(self):
        super().__init__()
        self.changed = False  # Flag to track if a change has been made
        global flag
        flag = False  # Ensure global flag is initialized properly

    def leave_BinaryOperation(self, original_node, updated_node):
        global flag  # Declare that we're working with the global 'flag'
        if not self.changed:

            # available_op_list = list(set(op_list).symmetric_difference(self.select_operations))
            available_op_list = list(set(self.select_operations) - set(op_list))
            if len(available_op_list) == 0:
              print("-----Available operations list is Empty-----")
              available_op_list = self.select_operations
              op_list.clear()

            current_op = random.choice(available_op_list)  # Get a random operation to check for
            # print(current_op)
            op_list.append(current_op)

            new_op_to_change = random.choice(self.operations)  # Pick a random new operation to replace
            if isinstance(updated_node.operator, current_op):  # Ensure the new operation is different
                self.changed = True  # Mark that a change has been made
                flag = True  # Set the global flag to True after making a change
                return updated_node.with_changes(
                    operator=new_op_to_change.with_changes(
                        whitespace_before=cst.SimpleWhitespace(value=''),
                        whitespace_after=cst.SimpleWhitespace(value='')
                    )
                )
        return updated_node

# Original code with multiple lines and parentheses
code = df['source'][2]
print(code)

# print(code)

while not flag:
  module = cst.parse_module(code)
  modified_tree = module.visit(OperationChanger())
  if not flag:
    print("No changes were made.")


print('\n')
c = modified_tree.code
print(c)

from decimal import *
import math
getcontext().prec=50


a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a+b)%2==0:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))
No changes were made.
-----Available operations list is Empty-----
No changes were made.
No changes were made.
-----Available operations list is Empty-----
No changes were made.


from decimal import *
import math
getcontext().prec=50


a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))


In [335]:
# Random changing value
import libcst as cst
import random

class NumberCollector(cst.CSTVisitor):
    def __init__(self):
        self.numbers = []

    def visit_Integer(self, node):
        self.numbers.append(node)

    def visit_Float(self, node):
        self.numbers.append(node)

class NumberModifier(cst.CSTTransformer):
    def __init__(self, target_node):
        super().__init__()
        self.target_node = target_node  # The specific node to change

    def leave_Integer(self, original_node, updated_node):
        if original_node == self.target_node:
            return updated_node.with_changes(value=str(int(original_node.value) + 100))  # Modify the chosen integer
        return updated_node

    def leave_Float(self, original_node, updated_node):
        if original_node == self.target_node:
            return updated_node.with_changes(value=str(float(original_node.value) - 100))  # Modify the chosen float
        return updated_node

# Original code as a string
code = """
a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0.5:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))
"""

# Parse the code
module = cst.parse_module(code)

# Step 1: Collect all numbers (integers and floats) from the code
collector = NumberCollector()
module.visit(collector)

# Step 2: Select a random number from the collected numbers
if collector.numbers:
    target_node = random.choice(collector.numbers)

    # Step 3: Create a transformer to modify the randomly chosen number
    modifier = NumberModifier(target_node)

    # Step 4: Apply the transformer to modify the code
    modified_tree = module.visit(modifier)

    # Print the modified code
    print(modified_tree.code)
else:
    print("No numbers found in the code.")



a,b=map(Decimal, input().split())
if b>a:
    print (-1)
else:
    cur=-1
    if (a|b)%2==0.5:
        cur=(a+b)//102
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))



In [341]:
# Changing decision
import libcst as cst
import random

class ComparisonCollector(cst.CSTVisitor):
    def __init__(self):
        self.comparisons = []

    def visit_Comparison(self, node):
        self.comparisons.append(node)

class SingleComparisonModifier(cst.CSTTransformer):
    def __init__(self, target_node):
        super().__init__()
        self.target_node = target_node  # The specific comparison node to change

    def leave_Comparison(self, original_node, updated_node):
        if original_node == self.target_node:
            # Randomly modify the comparison operators of the selected node
            modified_ops = []
            for operator in updated_node.comparisons:
                if isinstance(operator.operator, cst.LessThan):
                    # Change < to >
                    modified_ops.append(operator.with_changes(operator=cst.GreaterThan()))
                elif isinstance(operator.operator, cst.GreaterThan):
                    # Change > to ==
                    modified_ops.append(operator.with_changes(operator=cst.Equal()))
                elif isinstance(operator.operator, cst.Equal):
                    # Change == to <
                    modified_ops.append(operator.with_changes(operator=cst.LessThan()))
                # Add more modifications as needed for other operators
                else:
                    modified_ops.append(operator)  # No change
            # Return the updated node with modified operators
            return updated_node.with_changes(comparisons=modified_ops)
        return updated_node

# Original code as a string
code = """
a,b=map(Decimal, input().split())
if b>=0:
    cur=-1
    if flag:
        cur=(a+b)//2
    else:
        cur=(a+b)/Decimal(2)
    print(cur/math.floor(cur/b))
"""

# Parse the code
module = cst.parse_module(code)

# Step 1: Collect all comparisons
collector = ComparisonCollector()
module.visit(collector)

# Step 2: Select one random comparison node
if collector.comparisons:
    target_node = random.choice(collector.comparisons)

    # Step 3: Create a transformer to modify the chosen comparison operator
    modifier = SingleComparisonModifier(target_node)

    # Step 4: Apply the transformer to modify the code
    modified_tree = module.visit(modifier)

    # Print the modified code
    print(modified_tree.code)
else:
    print("No comparisons found in the code.")


AttributeError: module 'libcst' has no attribute 'LessThanOrEqual'

In [259]:
# Find witch line has changed
changed_lines = []
line = 1;
for original, mutated in zip(code.split('\n'), c.split('\n')):
    if original != mutated:
      print(f"Line {line} has changed")
      changed_lines.append(mutated)
    line += 1

# Output changed lines
if changed_lines:
    print("\n--- Changed Lines ---")
    for line in changed_lines:
        print(line)
else:
    print("No changes detected.")

Line 11 has changed

--- Changed Lines ---
    if (a-b)%2==0:
